### Important:
The data used in this exercise has already been cleaned. However, I included this notebook with a basic data cleaning step, as the process may vary depending on the context. For example, if the token is a stablecoin, it might be necessary to remove very small or very large values. Zero-value transactions could be relevant for identifying network activity, or they might indicate failed transactions or contract issues. Repeated values in short time spans may suggest arbitrage or testing behaviour. Therefore, the cleaning strategy should be adapted to the analysis goal.

In [5]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv('../data/token_transfers.csv')

output_file = '../data/cleaned/token_transfers_cleaned.csv'
os.makedirs(os.path.dirname(output_file), exist_ok=True)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5280131 entries, 0 to 5280130
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   block_number       int64  
 1   transaction_index  int64  
 2   from_address       object 
 3   to_address         object 
 4   time_stamp         int64  
 5   contract_address   object 
 6   value              float64
dtypes: float64(1), int64(3), object(3)
memory usage: 282.0+ MB


,block_number,transaction_index,from_address,to_address,time_stamp,contract_address,value
0,14669683,7,0xd30b438df65f4f788563b2b3611bd6059bff4ad9,0xda816e2122a8a39b0926bfa84edd3d42477e9efd,1651105815,0xdac17f958d2ee523a2206206994597c13d831ec7,18.670000
1,14669683,45,0x4941834ed1428089ee76252f6f9d767e800499b0,0x28c6c06298d514db089934071355e5743bf21d60,1651105815,0xdac17f958d2ee523a2206206994597c13d831ec7,10000.000000
2,14669683,46,0x2c1f9a20711e14f8484a41123e20d1b06858ebea,0x28c6c06298d514db089934071355e5743bf21d60,1651105815,0xdac17f958d2ee523a2206206994597c13d831ec7,9942.313005
3,14669683,47,0x7784c4f53aa5f03bd6d3ca670c9d9c887cc38cb0,0x3cd751e6b0078be393132286c442345e5dc49699,1651105815,0xdac17f958d2ee523a2206206994597c13d831ec7,234.364097
4,14669683,48,0x66589b8278470d58d4112f3d065de0f75734312e,0x28c6c06298d514db089934071355e5743bf21d60,1651105815,0xdac17f958d2ee523a2206206994597c13d831ec7,9800.000000


In [ ]:
import re

df_excluded = pd.DataFrame()

# standardizations (valid with etherium)
df['from_address'] = df['from_address'].str.lower()
df['to_address'] = df['to_address'].str.lower()
df['contract_address'] = df['contract_address'].str.lower()

# Basic function to validate ethereum address
def is_valid_eth_address(addr):
    return isinstance(addr, str) and re.fullmatch(r"0x[a-f0-9]{40}", addr) is not None

# Remove lines with missing data in essential columns
essential_cols = ['block_number', 'transaction_index', 'from_address', 'to_address', 'time_stamp', 'contract_address', 'value']
missing_rows = df[df[essential_cols].isnull().any(axis=1)]
df_excluded = pd.concat([df_excluded, missing_rows])
df = df.drop(missing_rows.index.tolist())

# Remove trasactions with value < 0
zero_value_rows = df[df['value'] < 0.0]
df_excluded = pd.concat([df_excluded, zero_value_rows])
df = df.drop(zero_value_rows.index.tolist())

# Remove invalid address
invalid_from = df[~df['from_address'].apply(is_valid_eth_address)]
invalid_to = df[~df['to_address'].apply(is_valid_eth_address)]
invalid_contract = df[~df['contract_address'].apply(is_valid_eth_address)]
invalid_rows = pd.concat([invalid_from, invalid_to, invalid_contract]).drop_duplicates()
df_excluded = pd.concat([df_excluded, invalid_rows])
df = df.drop(invalid_rows.index.tolist())

# Convert timestamp to datetime
df['time_stamp'] = pd.to_datetime(df['time_stamp'], unit='s')
df['date'] = df['time_stamp'].dt.date

# Summary
print(f"Total original: {len(df) + len(df_excluded)}")
print(f"Valids: {len(df)}")
print(f"Excludeds: {len(df_excluded)}")
print(f"Tax of validation: {(len(df) / (len(df) + len(df_excluded)) * 100):.2f}%")
df_excluded.head()

In [ ]:
# Basic descriptive statistics
print("Descriptive:")
print(df['value'].describe())
print(f"\nPerccentiles: {df['value'].quantile([0.001, 0.01, 0.05, 0.95, 0.99, 0.999])}")

# Identify imposible values
test_values = df[df['value'] >= 1e15]
print(f"\nExtreme values (>= 1e15): {len(test_values)}")

# Analisys per contract
print("\nAnalisys per contract:")
for contract in df['contract_address'].value_counts().head(5).index:
    contract_data = df[df['contract_address'] == contract]
    print(f"{str(contract)[:10]}...: min={contract_data['value'].min():.4f}, max={contract_data['value'].max():.4f}")

# Remove outliers
q_high = df['value'].quantile(0.9999) # 99.99%
q_low = df['value'].quantile(0.0001) # 0.01%

outlier_high = df[df['value'] > q_high]
outlier_low = df[df['value'] < q_low]

print(f"\nHighest outliers (> {q_high:.2f}): {len(outlier_high)}")
print(f"Lowest outliers (< {q_low:.2f}): {len(outlier_low)}")

# Remove and update tracking
df_excluded = pd.concat([df_excluded, outlier_high, outlier_low])
df = df[(df['value'] >= q_low) & (df['value'] <= q_high)]

print(f"\nAfter outliers removal: {len(df)} valid transactions")

Direct links for mapping


USDT (Tether): https://coinmarketcap.com/currencies/tether/

USDC (USD Coin): https://coinmarketcap.com/currencies/usd-coin/

DAI: https://coinmarketcap.com/currencies/multi-collateral-dai/

UST (TerraClassicUSD): https://coinmarketcap.com/currencies/terrausd/

PAX (Pax Dollar): https://coinmarketcap.com/currencies/paxos-standard/

WLUNA (Wrapped LUNA Classic): https://coinmarketcap.com/currencies/wrapped-terra/

In [ ]:
# Map contract address to token names
contract_to_token = {
    '0xdac17f958d2ee523a2206206994597c13d831ec7': 'USDT',
    '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': 'USDC',
    '0x6b175474e89094c44da98b954eedeac495271d0f': 'DAI',
    '0xa47c8bf37f92abed4a126bda807a7b7498661acd': 'UST',
    '0x8e870d67f660d95d5be530380d0ec0bd388289e1': 'PAX',
    '0xd2877702675e6ceb975b4a1dffd6ea4a5f3e6797': 'WLUNA'
}

df['token'] = df['contract_address'].map(contract_to_token)
df = df[~df['token'].isnull()]

df.to_csv(output_file)
